In [3]:
import numpy as np
from math import sqrt,exp

In [4]:
S0 = 100     # spot price of the underlying stock today
K = 105      # strike at expiry
sigma = 0.25  # volatility
r = 0.02     # risk-free rate
T = 2/12      # years to expiry

In [8]:
# Lattice Parameter Calclations
lattice_unit_time = 1/52
u = exp(sigma * sqrt(lattice_unit_time))
d = 1/u
p = (np.exp(r*lattice_unit_time)-d)/(u-d) # Since expected return is assumed to be zero


1.0352767292790188


0.49688063503168617

In [4]:
def avg(list):
    return sum(list) / len(list)

# Uses recursion to explore all possible branches of lattice, and then adds their contributions to the option values
def traverse_lattice_options(option_values, time, path, probability):
    time += lattice_unit_time
    if time > T: 
        option_values['asian_call_value'] += max(0, avg(path) - K) * probability
        option_values['asian_put_value'] += max(0, K - avg(path)) * probability

        option_values['lookback_call_value'] += max(0, max(path) - K) * probability
        option_values['lookback_put_value'] += max(0, K - min(path)) * probability
        
        option_values['floating_lookback_call_value'] += max(0, path[-1] - min(path)) * probability
        option_values['floating_lookback_put_value'] += max(0, max(path) - path[-1]) * probability
    else: 
        path.append(path[-1] * u)
        traverse_lattice_options(option_values, time, path, probability*p)
        path.pop()
        
        path.append(path[-1] * d)
        traverse_lattice_options(option_values, time, path, probability*(1-p))
        path.pop()

# Wrapper function to get option values
def get_option_values():
    # Initialization
    option_values = {'asian_call_value' : 0, 
                     'asian_put_value' : 0,
                     'lookback_call_value' : 0,
                     'lookback_put_value' : 0,
                     'floating_lookback_call_value' : 0,
                     'floating_lookback_put_value' : 0}    
    
    traverse_lattice_options(option_values, 0, [S0], 1)    
    # Normalizing option cash flows by 2-month discount rate
    for option_type in option_values:        
        option_values[option_type] /= pow(1+r, 1/6)
    
    return option_values

In [5]:
print (get_option_values())

{'asian_call_value': 0.6425291611345371, 'asian_put_value': 5.386101289152401, 'lookback_call_value': 3.3923131529403463, 'lookback_put_value': 10.934321028544048, 'floating_lookback_call_value': 6.431038426246864, 'floating_lookback_put_value': 6.211902812425354}
